We can find the reduced form equivalent to a given form $f$ by reduction. If $f$ is not reduced then $c < a$, or $|b| > a$, or $a = -b$, or $a = c$ and $b < 0$. We define operations $S$, $T$ and $T^{-1}$ on forms by
\begin{align}
    S : (a, b, c) &\mapsto (c, −b, a), \\
    T : (a, b, c) &\mapsto (a, b + 2a, a + b + c), \\
    T^{1}: (a, b, c) &\mapsto (a, b − 2a, a − b + c).
\end{align}

These operations are represented by matrices
\begin{equation}
    S =
    \begin{pmatrix}
        0 & -1 \\
        1 & 0
    \end{pmatrix},
    \quad T =
    \begin{pmatrix}
        1 & 1 \\
        0 & 1
    \end{pmatrix}
\end{equation}
in $SL_2(\mathbb{Z})$, so that each operation yields an equivalent form. If a form is not reduced, then one of these operations may be applied and the result will be closer to a reduced form as $|a| + |b|$ is made smaller.

*   Size reduction: If $a > c$, or if $a = c$ and $b < 0$, then the form is transformed using the $S$ operation. This ensures that the first coefficient $a$ is the smallest of the outer coefficients and handles a specific boundary case.
*   Coefficient reduction: If $b > a$, then $T^{-1}$ is applied $n$ times, while if $b \leq -a, then $T$ is applied $n$ times for some specified number $n$.

In [41]:
def is_reduced(a, b, c):
    '''
    Checks if a positive definite form (a, b, c) is reduced.
    A form is reduced if (-a < b <= a < c) or (0 <= b <= a = c).
    '''
    return (-a < b <= a < c) or (0 <= b <= a == c)

def reduce_quadratic_form(a_in, b_in, c_in):
    '''
    Reduces a positive definite binary quadratic form (a, b, c) and
    returns the reduced form along with the sequence of operations.
    Args:
        a_in: The 'a' coefficient of the form.
        b_in: The 'b' coefficient of the form.
        c_in: The 'c' coefficient of the form.
    Returns:
        A tuple containing the reduced form (a, b, c) and a list
        of the operations performed (e.g., ['S', 'T', 'T']).
    '''
    a, b, c = a_in, b_in, c_in
    operations = []

    # Loop until the form is in a reduced state.
    while not is_reduced(a, b, c):

        # Size reduction
        # If a > c, or if a = c and b is negative, then apply S to swap a and c.
        if a > c or (a == c and b < 0):
            a, b, c = c, -b, a
            operations.append('S')
            # After applying S, b might be out of range, so we continue the loop
            # to re-evaluate from the top.
            continue

        # Coefficient reduction
        # If we reach this point, then we know thta a <= c.
        # Now, we must bring b into the range (-a, a].

        if abs(b) > a:
            if b > a:
                # Apply T^-1 n times to reduce b.
                # The formula for T^-n is: (a, b-2an, an^2-bn+c)
                n = (b + a - 1) // (2 * a)
                c = a*n*n - b*n + c
                b = b - 2*a*n
                operations.extend(['T^-1'] * n)

            elif b <= -a:
                # Apply T^n n times to increase b.
                # The formula for T^n is: (a, b+2an, an^2+bn+c)
                n = (-b + a) // (2*a)
                c = a*n*n + b*n + c
                b = b + 2*a*n
                operations.extend(['T'] * n)

    return (a, b, c), operations

forms_to_reduce = [
    (220, 594, 401),
    (226, 367, 149)
]

for form in forms_to_reduce:
    print(f"\nReducing form {form}...")

    reduced_form, op_sequence = reduce_quadratic_form(*form)

    print(f"  Reduced Form: {reduced_form}")
    print(f"  Operations: {', '.join(op_sequence)}")


Reducing form (220, 594, 401)...
  Reduced Form: (1, 0, 11)
  Operations: T^-1, S, T, T, T, S, T, T, T, T

Reducing form (226, 367, 149)...
  Reduced Form: (1, 1, 2)
  Operations: S, T, S, T^-1, T^-1, T^-1, T^-1, S, T, T, T


The composition of primitive forms $f_1 = (a_1, b_1, c_1)$ and $f_2 = (a_2, b_2, c_2)$, with the same discriminant $d$, is defined as follows. First we set
\begin{equation}
    \beta = \frac{b_1 + b_2}{2}, \quad \gamma = \frac{b_2 - b_1}{2}.
\end{equation} Then we use the Euclidean algorithm twice. The first time we compute $m = \gcd(a_1, \beta)$ and find integers $x$ and $y$ with $a_1x + \beta y = m$. The second time we compute $n = \gcd(m, a_2)$ and solve the congruence
\begin{equation}
    \frac{m}{n}z \equiv \gamma x − c_1y \mod \frac{a_2}{n}
\end{equation}
for $z$. The composition of $f_1$ and $f_2$ is then
\begin{equation}
    f_3 = f_1 \circ f_2 = \left(\frac{a_1a_2}{n_2}, b_1 + \frac{2a_1z}{n}, ∗\right)
\end{equation}
where the third coefficient $*$ is chosen so that $f_3$ also has discriminant $d$.

To solve for $z$, we need to find a modular multiplicative inverse which is guaranteed to exist, as $\gcd(m/n, a_2n) = 1$. This is a direct consequence of $n = \gcd(m, a_2)$ and can be solved using the extended Euclidean algorithm and Bezout's identity.

It is known that if $f_1 \sim g_1$ and $f_2 \sim g_2$ then $f_1 \circ f_2 \sim g_1 \circ g_2$.


In [42]:
import math

def extended_gcd(a, b):
    '''
    Extended euclidean algorithm to find g, x, y such that ax + by = g.
    Used for solving Bezout's identity and for modular inverse.
    '''
    if a == 0:
        return (b, 0, 1)
    g, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return (g, x, y)

def modular_inverse(a, m):
    '''
    Finds the modular multiplicative inverse of a modulo m.
    Uses the extended euclidean algorithm.
    '''
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    return x % m

def compose_forms(f1, f2):
    '''
    Computes the composition of two primitive quadratic forms f1 and f2
    with the same discriminant.
    '''
    a1, b1, c1 = f1
    a2, b2, c2 = f2

    # Check if discriminants match
    d = b1**2 - 4*a1*c1
    if d != b2**2 - 4*a2*c2:
        raise ValueError("Forms must have the same discriminant.")

    # Calculate beta and gamma
    beta = (b1 + b2) // 2
    gamma = (b2 - b1) // 2

    # First Euclidean algorithm
    m, x, y = extended_gcd(a1, beta)

    # Second Euclidean algorithm
    n = math.gcd(m, a2)

    # Solve the congruence for z
    # (m/n)z ≡ (gamma*x - c1*y) (mod a2/n)
    A = m // n
    B = gamma * x - c1 * y
    M = a2 // n

    # Find the modular inverse
    A_inv = pow(A, -1, M)
    z = (B * A_inv) % M

    # Construct the new form f3 = (a3, b3, c3)
    a3 = (a1 * a2) // (n**2)
    b3 = b1 + (2 * a1 * z) // n
    c3 = (b3**2 - d) // (4 * a3)

    return (a3, b3, c3)

def reduce_form(f_in):
    '''
    Reduces a quadratic form to its unique equivalent reduced form.
    '''
    a, b, c = f_in
    while True:
        # Condition for a reduced form: -a < b <= a <= c
        if -a < b <= a and a < c:
            break
        if a == c and 0 <= b <= a:
            break

        # Apply S if necessary
        if a > c or (a == c and b < 0):
            a, b, c = c, -b, a
            continue

        # Apply T^n or T^-n to bring b into range
        if b > a:
            n = (b + a - 1) // (2*a)
            b, c = b - 2*a*n, a*n*n - b*n + c
        elif b <= -a:
            n = (-b + a) // (2*a)
            b, c = b + 2*a*n, a*n*n + b*n + c
    return (a, b, c)

In [43]:
f1 = (2, 1, 3)
f2 = (2, 1, 3)
print(f"\nComposing f1 = {f1} and f2 = {f2} (d = -23).")
composed = compose_forms(f1, f2)
reduced_composed = reduce_form(composed)
print(f"  Composed form: {composed}")
print(f"  Reduced equivalent: {reduced_composed}")

f1 = (2, 1, 3)
f2 = (2, 1, 3)
g1 = (2, -3, 4)
g2 = (3, -1, 2)
print(f"\nTest Case 1: d = -23")
print(f"  f1 = {f1}, g1 = {g1} (g1 ~ f1)")
print(f"  f2 = {f2}, g2 = {g2} (g2 ~ f2)")
comp1 = compose_forms(f1, f2)
reduced1 = reduce_form(comp1)
print(f"  f1 o f2 = {comp1}, which reduces to {reduced1}")
comp2 = compose_forms(g1, g2)
reduced2 = reduce_form(comp2)
print(f"  g1 o g2 = {comp2}, which reduces to {reduced2}")
print(f"  Property holds: {reduced1 == reduced2}")

print("\nTest Case: d = -104")
f1 = (3, 2, 9)
f2 = (5, 4, 6)
g1 = (3, -4, 10)
g2 = (5, 4, 6)
print(f"  f1 = {f1} (d={f1[1]**2 - 4*f1[0]*f1[2]})")
print(f"  g1 = {g1} (d={g1[1]**2 - 4*g1[0]*g1[2]}) -> g1 ~ f1")
print(f"  f2 = {f2} (d={f2[1]**2 - 4*f2[0]*f2[2]})")
print(f"  g2 = {g2} (d={g2[1]**2 - 4*g2[0]*g2[2]}) -> g2 ~ f2")
comp1 = compose_forms(f1, f2)
reduced1 = reduce_form(comp1)
print(f"  f1 o f2 = {comp1}, which reduces to {reduced1}")
comp2 = compose_forms(g1, g2)
reduced2 = reduce_form(comp2)
print(f"  g1 o g2 = {comp2}, which reduces to {reduced2}")
print(f"  -> Property holds: {reduced1 == reduced2}")


Composing f1 = (2, 1, 3) and f2 = (2, 1, 3) (d = -23).
  Composed form: (4, 5, 3)
  Reduced equivalent: (2, -1, 3)

Test Case 1: d = -23
  f1 = (2, 1, 3), g1 = (2, -3, 4) (g1 ~ f1)
  f2 = (2, 1, 3), g2 = (3, -1, 2) (g2 ~ f2)
  f1 o f2 = (4, 5, 3), which reduces to (2, -1, 3)
  g1 o g2 = (6, 5, 2), which reduces to (2, -1, 3)
  Property holds: True

Test Case: d = -104
  f1 = (3, 2, 9) (d=-104)
  g1 = (3, -4, 10) (d=-104) -> g1 ~ f1
  f2 = (5, 4, 6) (d=-104)
  g2 = (5, 4, 6) (d=-104) -> g2 ~ f2
  f1 o f2 = (15, 14, 5), which reduces to (5, -4, 6)
  g1 o g2 = (15, 14, 5), which reduces to (5, -4, 6)
  -> Property holds: True


Let $d$ be a discriminant, i.e., a negative integer that is congruent to $0$ or $1$ modulo $4$. It is known that the set of equivalence classes of primitive binary quadratic forms of discriminant $d$ forms an abelian group under composition, known as the class group. The identity class contains either $(1, 0, -d/4)$ or $(1, 1,(1 - d)/4)$. The inverse of the class containing $(a, b, c)$ is the class of $(a, -b, c)$.

The fundamental theorem of finitely generated abelian groups states that every (non-trivial) finite abelian group may uniquely be written in the form
\begin{equation}
    C_{n_1} \times \cdots \times C_{n_t},
\end{equation}
where $n_1, \dots, n_t$ are integers greater than one with $n_1| \cdots |n_t$.

The elements of the class group are the equivalence classes of primitive binary quadratic forms which are represented by the set of primitive reduced forms. First, we generate all primitive reduced forms for a given $d$ where the total number of these forms is the class number $h(d)$ which is also the order of the glass group.

To distinguish between groups of the same order we can count the number of elements of each order. The number of elements of order $2$ is particularly easy to find and can be very powerful for this purpose. An class has order $1$ or $2$ if and only if its inverse is itself. For a reduced form, this means that $b = 0$, $b = a$, or $a = c$. Subtracting $1$ for the identity gives the number of elements of order exactly $2$.

The class number and the classes of order $2$ is often enough to deduce the group structure.
*   If $h(d) = p$ is prime, then the class roup is cyclic $C_p$.
*   If $h(d) = p^2$ is a prime squared, then:
    *   One element of order $2$ implies the group is $C_4$.
    *   Three elements of order $2$ implies the group is $K_4$.
*   If $h(d) = p^3$, then:
    *   One element of order $2$ implies $C_8$.
    *   Three elements of order $2$ implies $C_4 \times C_2$.
    *   Seven elements of order $2$ implies $C_2 \times C_2 \times C_2$.
*   If $h = pq$ is a product of distinct primes, then the group is $C_{pq} \cong C_p \times C_q$.